<a href="https://colab.research.google.com/github/fengfrankgthb/Demonstrations/blob/main/2025_SAT_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAT HHH tested w/ all-mpnet-base-v2 Model 2025.05.28

In this project, an well-trained **transformer model** is used to represent an knowledgable super-student in front of SAT and GRE reading quexstions. **Biases** and **confusions** are disclosed, likely attributable to **over-fitting** to the training of the model. Over-fitting is the ML terminology for **test cramming**, a phenominon when fitting specificities of **training dataset** caused the model to not being able to fit to specificities of **testing dataset**.

## 1. Install Necessary Libraries
* **sentence-transformers** This is the text embedding library
* **scikit-learn** This is the machine learning library
* **matplotlib** This is the mat-lab style plotting library

In [1]:
!pip install sentence-transformers scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## 2. import necessary modules from the libraries



**mpl_toolkits.mplot3d**: matplot 3D plotting lib

**numpy**: Numerical Python, the fundamental python lib

**Axes3D** 3D plotting class

**PCA** Principal Components Analysis for linear dimension reduction.

**TSNE** t-SNE non-linear dimension reduction to creat more scattered effect

In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

# set matplot to inline (static) mode, or notebook for interactive mode)
# even though default is inline mode, be explicit to avoid any confusion
# the interactive notebook mode is often unstable at colab environment
# alternative is set as 'inline' and use 'plotly'.
%matplotlib inline

## 3. Imput text data

Choose one subsection below:

### 3.1 Example 1: Aristotle Politics, All-mighty's age_intel vs 5Vs

Used **248HHH Q170** on **Aritotle Politics** for illustration, breaking into 9 components:

* Pa = All Sentences combined in Passage
* P1 = 1st sentence in Passage
* P2 = 2nd sentence in Passage
* P3 = 3rd sentence in Passage
* Q? = the Question sentence
* Av = wrong choice A
* Bx = correct answer B
* Cx = wrong choice C
* Dx = wrong choice D

In this example of old text, we discover that an advantage of an well-knowledgable person, **age_intel** as it's dubbed here. Age_intel come from training experience by a large amount of text data, like the 10Billion words comprehended by All-mighty Model. This is a level of intelligence reachable by few individuals, thus should **NOT** be your objective in test preparation.

Surprisingly, using Unified Moodels relying on bold 5Vs, it takes you below a minute to spot the correct answer out of wrong choices, each of which has a redundant V irrelevant to the prompt itself.

In [3]:
# Example 1: Aristotle Politics Question 347Words
# This is a 2024-25 **** question used to discover the age_intel of All-mighty old text.
# All-mighty, or a Englkish language well-knowledgable person may have an advantage, unreachable by school smartness.
# This is called age_intel.
sentences = ["Pa: A student in a political science course is writing a paper on Aristotle’s The Politics, in which Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved. Aristotle observes that different forms of government can fall in diferent ways--for example, oligarchies might grant power to military leaders during wartime who refuse to relinquish that power during peacetime--but some methods of preserving order apply across all forms of government. The student claims that in particular Aristotle asserts that in a healthy state obedience to law must be as close to absolute as possible and that even minor infractions should not be ignored.",
    "P1: A student in a political science course is writing a paper on Aristotle’s The Politics, in which Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved.",
    "P2: Aristotle observes that different forms of government can fall in diferent ways--for example, oligarchies might grant power to military leaders during wartime who refuse to relinquish that power during peacetime--but some methods of preserving order apply across all forms of government.",
    "P3: The student claims that in particular Aristotle asserts that in a healthy state obedience to law must be as close to absolute as possible and that even minor infractions should not be ignored. ",
    "Q?: Which quotation from a philosopher's analysis of The Politics would best support the student's claim?",
    "Av) When constructing his argument regarding the characteristics of a well-functioning government, Aristotle asserts that ‘Transgression creeps in unperceived and at last ruins the state,’ illustrating this idea with a comparison to frequent small expenditures slowly and almost imperceptibly chipping away at a fortune until it is ultimately depleted.",
    "Bx) When Aristotle considers the health of constitutions, he states that ‘Constitutions are preserved when their destroyers are at a distance, and sometimes also because they are near, for the fear of them makes the government keep in hand the constitution.' He holds that rulers who wish to see constitutions preserved must continually remind the populace of the dangers that would result from a constitutional collapse.",
    "Cx) When contrasting different forms of government, Aristotle holds that ‘oligarchies may last, not from any inherent stability in such forms of government, but because the rulers are on good terms both with the unenfranchised and with the governing classes,’ That is, oligarchic leaders who wish to hold on to power will introduce members of disenfranchised classes into government in a participatory role.",
    "Dx) When Aristotle writes on the necessity of avoiding corruption in government, he proposes that 'every state should be so administered and so regulated by law that its magistrates cannot possibly make money.' In particular he thinks oligarchies are particularly susceptible to corruption through bribery."
]

In [6]:
# Example 1 (Alt-1): Aristotle Politics Question 347Words
# This is 1st alteration to discover the inability of long_comp_conf in old text.
sentences = ["Pa: A student in a political science course is writing a paper on Aristotle’s The Politics, in which Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved. Aristotle observes that different forms of government can fall in diferent ways--for example, oligarchies might grant power to military leaders during wartime who refuse to relinquish that power during peacetime--but some methods of preserving order apply across all forms of government. The student claims that in particular Aristotle asserts that in a healthy state obedience to law must be as close to absolute as possible and that even minor infractions should not be ignored.",
    "P1: In The Politics, Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved.",
    "P2: Aristotle observes that government can fall in different forms, but some methods of preserving order apply across all forms of government.",
    "P3: Aristotle asserts that obedience to law must be absolute and that even minor infractions should not be ignored.",
    "Q?: Which quotation from a philosopher's analysis of The Politics would best support this assertion?",
    "Av) When constructing his argument regarding the characteristics of a well-functioning government, ‘Transgression creeps in unperceived and at last ruins the state,’ illustrating this idea with a comparison to frequent small expenditures slowly and almost imperceptibly chipping away at a fortune until it is ultimately depleted.",
    "Bx) When Aristotle considers the health of constitutions, he states that ‘Constitutions are preserved when their destroyers are at a distance, and sometimes also because they are near, for the fear of them makes the government keep in hand the constitution.' He holds that rulers who wish to see constitutions preserved must continually remind the populace of the dangers that would result from a constitutional collapse.",
    "Cx) When contrasting different forms of government, Aristotle holds that ‘oligarchies may last, not from any inherent stability in such forms of government, but because the rulers are on good terms both with the unenfranchised and with the governing classes,’ That is, oligarchic leaders who wish to hold on to power will introduce members of disenfranchised classes into government in a participatory role.",
    "Dx) When Aristotle writes on the necessity of avoiding corruption in government, he proposes that 'every state should be so administered and so regulated by law that its magistrates cannot possibly make money.' In particular he thinks oligarchies are particularly susceptible to corruption through bribery."
]

In [9]:
# Example 1 (Alt-2): Aristotle Politics Question 347Words
# This is 2nd alteration to discover the inability of long_comp_unconf in old text.
sentences = ["Pa: A student in a political science course is writing a paper on Aristotle’s The Politics, in which Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved. Aristotle observes that different forms of government can fall in diferent ways--for example, oligarchies might grant power to military leaders during wartime who refuse to relinquish that power during peacetime--but some methods of preserving order apply across all forms of government. The student claims that in particular Aristotle asserts that in a healthy state obedience to law must be as close to absolute as possible and that even minor infractions should not be ignored.",
    "P1: In The Politics, Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved.",
    "P2: Aristotle observes that government can fall in different forms, but some methods of preserving order apply across all forms of government.",
    "P3: Aristotle asserts that obedience to law must be absolute and that even minor infractions should not be ignored.",
    "Q?: Which quotation from a philosopher's analysis of The Politics would best support this assertion?",
    "Av) 'Transgression creeps in unperceived and at last ruins the state': this idea with a comparison to frequent small expenditures slowly chipping away at a fortune until it is ultimately depleted.",
    "Bx) 'Constitutions are preserved when their destroyers are at a distance, and sometimes also because they are near, for the fear of them makes the government keep in hand the constitution': rulers must continually remind the populace of the dangers from a constitutional collapse.",
    "Cx) 'Oligarchies may last, not from any inherent stability in such forms of government, but because the rulers are on good terms both with the unenfranchised and with the governing classes': oligarchic leaders will introduce disenfranchised classes into participatory roles.",
    "Dx) 'Every state should be so administered and so regulated by law that its magistrates cannot possibly make money': oligarchies are particularly susceptible to corruption through bribery."
]

In [35]:
# Example 1 (Alt-3): Aristotle Politics Question 347Words
# This is 3rd alrteration to discover the inability of long_Comp_unconf in old text.
sentences = ["Pa: A student in a political science course is writing a paper on Aristotle’s The Politics, in which Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved. Aristotle observes that different forms of government can fall in diferent ways--for example, oligarchies might grant power to military leaders during wartime who refuse to relinquish that power during peacetime--but some methods of preserving order apply across all forms of government. The student claims that in particular Aristotle asserts that in a healthy state obedience to law must be as close to absolute as possible and that even minor infractions should not be ignored.",
    "P1: In The Politics, Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved.",
    "P2: Aristotle observes that government can fall in different forms, but some methods of preserving order apply across all forms of government.",
    "P3: Aristotle asserts that obedience to law must be absolute and that even minor infractions should not be ignored.",
    "Q?: Which quotation from a philosopher's analysis of The Politics would best support this assertion?",
    "Av) frequent small expenditures slowly chipping away at a fortune until it is ultimately depleted.",
    "Bx) rulers must continually remind the populace of the dangers from a constitutional collapse.",
    "Cx) oligarchic leaders will introduce disenfranchised classes into participatory roles.",
    "Dx) oligarchies are particularly susceptible to corruption through bribery."
]

In [23]:
# Example 1 (Alt-4): Aristotle Politics Question 347Words
# This is 4th alteration to discover the long_Comp_unconf in old text.
sentences = ["Pa: A student in a political science course is writing a paper on Aristotle’s The Politics, in which Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved. Aristotle observes that different forms of government can fall in diferent ways--for example, oligarchies might grant power to military leaders during wartime who refuse to relinquish that power during peacetime--but some methods of preserving order apply across all forms of government. The student claims that in particular Aristotle asserts that in a healthy state obedience to law must be as close to absolute as possible and that even minor infractions should not be ignored.",
    "P1: In The Politics, Aristotle offers his opinion on political instability and gives advice on how constitutions can be preserved.",
    "P2: Aristotle observes that government can fall in different forms, but some methods of preserving order apply across all forms of government.",
    "P3: Aristotle asserts that obedience to law must be absolute and that even minor infractions should not be ignored.",
    "Q?: Which quotation from a philosopher's analysis of The Politics would best support this assertion?",
    "Av) 'Transgression creeps in unperceived and at last ruins the state.'",
    "Bx) 'Constitutions are preserved when their destroyers are at a distance, and sometimes also because they are near, for the fear of them makes the government keep in hand the constitution.'",
    "Cx) 'Oligarchies may last, not from any inherent stability in such forms of government, but because the rulers are on good terms both with the unenfranchised and with the governing classes.'",
    "Dx) 'Every state should be so administered and so regulated by law that its magistrates cannot possibly make money.'"
]

In [26]:
# Example 1 (Alt-4): Aristotle Politics Question 347Words
# This is 5th alreration to discover the inertia of age_intel in working with 5Vs.
sentences = ["Pa: subjective negative; many one; positive; negative negative.",
    "P1: subjective; negative; positive.",
    "P2: subjective; negative many; but positive one.",
    "P3: subjective; positive positive; negative negative.",
    "Q?: subjective; positive?",
    "Av) subjective; many negative negative.",
    "Bx) subjective; positive negative; cause effect",
    "Cx) subjective; positive.",
    "Dx) subjective; negative; cause effect."
]

## 4. Generate Embeddings for Aristotle Sentences.


**So, Let run the embedding code below** ↓ ↓ ↓

In [36]:
# Load the pre-trained model
model = SentenceTransformer('all-mpnet-base-v2')

# Generate the embeddings
embeddings = model.encode(sentences)

print("Shape of embeddings:", embeddings.shape)
print("Example embedding (last sentence):\n", embeddings[8][:100]) # Print the first 100 dimensions only

Shape of embeddings: (9, 768)
Example embedding (last sentence):
 [ 5.8507686e-03  4.8937615e-02 -8.4935622e-03  2.4388717e-02
  4.2667878e-03  7.6053850e-03 -1.1713018e-02 -2.5614321e-02
  2.6843412e-02  4.1981213e-02 -1.8735992e-02  5.0152391e-02
  3.4754969e-02 -2.1343140e-03  4.9161918e-02 -4.3324824e-02
  3.8703520e-02 -1.0086688e-02 -1.9598873e-02 -2.8380724e-02
  1.1832624e-02 -2.1399764e-02  3.3981014e-02 -4.2395137e-02
  3.6291528e-02  1.9450083e-02  6.0230121e-02 -7.5430577e-03
 -5.2264526e-02  5.0232310e-02  3.4364372e-02  3.1964377e-02
  1.1038023e-02 -5.7826671e-03  1.5988381e-06 -1.9332042e-02
 -5.8815185e-02 -1.6528862e-02  4.5572519e-03  4.5355216e-02
 -1.2939898e-02 -1.0109472e-03 -1.0969699e-02 -7.3170546e-03
 -7.6917922e-03 -1.0006217e-02 -1.7978504e-02 -6.4541702e-04
  4.8660856e-02  1.6099209e-04  5.8345492e-03  1.1514657e-01
 -7.6134861e-03 -2.8412428e-02 -2.5363263e-02 -7.8299515e-02
  1.0392533e-02  1.2263651e-02 -9.9089712e-02  4.7988478e-02
  1.1608282e-03 -1.

# 5. Reduce Dimensionality: 3D with PCAand P-sphere, P-tet, and Scatterplot

This is the **primary demonstration** of analytical result, showing the question elements (**prompt**, **question**, and **choices**) in all three renderings (**sphere**, **tetrehedron**, and **scatterplots**)

In [37]:
# Assumed Embeddings from section 4.Generate Embeddings for Sentences.
# Sphere, tetrahedron, and scatterplot.
import plotly.graph_objects as go
import numpy as np
from sklearn.decomposition import PCA

labels = [s[:2] for s in sentences]  # Get first two letters.

# Perform PCA to reduce to 3 dimensions
pca_3d = PCA(n_components=3)
reduced_embeddings_3d = pca_3d.fit_transform(embeddings)

# Directly define the indices, assuming P1, P2, P3, Q? are at indices 1, 2, 3, and 4
p1_index = 1
p2_index = 2
p3_index = 3
q_index = 4

# Extract the 3D coordinates of P1, P2, P3, and Q?
p1_coords = reduced_embeddings_3d[p1_index]
p2_coords = reduced_embeddings_3d[p2_index]
p3_coords = reduced_embeddings_3d[p3_index]
q_coords = reduced_embeddings_3d[q_index]

# Define the vertices of the tetrahedron
tetrahedron_vertices = np.array([p1_coords, p2_coords, p3_coords, q_coords])

# Define the edges of the tetrahedron (indices of vertices)
tetrahedron_edges = [
    (0, 1), (0, 2), (0, 3),  # Edges from P1 to P2, P3, Q?
    (1, 2), (1, 3),          # Edges from P2 to P3, Q?
    (2, 3)                   # Edge from P3 to Q?
]

# Create the lines for the tetrahedron edges
lines = []
for i, (start_index, end_index) in enumerate(tetrahedron_edges):
    start_point = tetrahedron_vertices[start_index]
    end_point = tetrahedron_vertices[end_index]
    lines.append(
        go.Scatter3d(
            x=[start_point[0], end_point[0]],
            y=[start_point[1], end_point[1]],
            z=[start_point[2], end_point[2]],
            mode='lines',
            line=dict(color='red', width=4),  # Style the lines
            name=f'Edge {i+1}'
        )
    )

# --- Start of new code for sphere calculation ---
# Points for sphere calculation
P = [p1_coords, p2_coords, p3_coords, q_coords]
x = [p[0] for p in P]
y = [p[1] for p in P]
z = [p[2] for p in P]

# System of equations to find sphere center (cx, cy, cz)
# (xi-cx)^2 + (yi-cy)^2 + (zi-cz)^2 = R^2
# Subtracting equation for P1 from P2, P3, P4:
# 2(x2-x1)cx + 2(y2-y1)cy + 2(z2-z1)cz = (x2^2+y2^2+z2^2) - (x1^2+y1^2+z1^2)
# ... and so on for P3 and P4

A = np.array([
    [2*(x[1]-x[0]), 2*(y[1]-y[0]), 2*(z[1]-z[0])],
    [2*(x[2]-x[0]), 2*(y[2]-y[0]), 2*(z[2]-z[0])],
    [2*(x[3]-x[0]), 2*(y[3]-y[0]), 2*(z[3]-z[0])]
])

B = np.array([
    (x[1]**2 + y[1]**2 + z[1]**2) - (x[0]**2 + y[0]**2 + z[0]**2),
    (x[2]**2 + y[2]**2 + z[2]**2) - (x[0]**2 + y[0]**2 + z[0]**2),
    (x[3]**2 + y[3]**2 + z[3]**2) - (x[0]**2 + y[0]**2 + z[0]**2)
])

try:
    sphere_center = np.linalg.solve(A, B)
    cx, cy, cz = sphere_center[0], sphere_center[1], sphere_center[2]
    sphere_radius = np.sqrt((x[0]-cx)**2 + (y[0]-cy)**2 + (z[0]-cz)**2)

    # Generate sphere surface points
    u = np.linspace(0, 2 * np.pi, 50) # Azimuthal angle
    v = np.linspace(0, np.pi, 25)    # Polar angle

    sphere_x = cx + sphere_radius * np.outer(np.cos(u), np.sin(v))
    sphere_y = cy + sphere_radius * np.outer(np.sin(u), np.sin(v))
    sphere_z = cz + sphere_radius * np.outer(np.ones(np.size(u)), np.cos(v)) # Corrected from np.ones_like(u) to np.ones(np.size(u))

    sphere_surface = go.Surface(
        x=sphere_x, y=sphere_y, z=sphere_z,
        opacity=0.3,
        colorscale='Blues', # You can choose other colorscales e.g. 'Viridis', 'RdBu'
        showscale=False, # Hide the color scale bar for the sphere
        name='Sphere'
    )
    sphere_added = True
except np.linalg.LinAlgError:
    print("Could not determine the sphere: Points might be coplanar or collinear.")
    sphere_surface = None # No sphere to add if calculation fails
    sphere_added = False
# --- End of new code for sphere calculation ---

# Create the 3D scatter plot and add the lines and sphere
data_elements = [go.Scatter3d(
    x=reduced_embeddings_3d[:, 0],
    y=reduced_embeddings_3d[:, 1],
    z=reduced_embeddings_3d[:, 2],
    mode='markers+text',
    text=labels,  # Use first 2 characters of labels
    textposition="middle right",
    marker=dict(size=4),
    name='Data Points'
)] + lines

if sphere_added and sphere_surface:
    data_elements.append(sphere_surface)

fig = go.Figure(data=data_elements)


# Set the title and axis labels
fig.update_layout(
    title="3D Scatterplot, P-Tet and Sphere",
    scene=dict(
        xaxis_title="PC 1",
        yaxis_title="PC 2",
        zaxis_title="PC 3",
        # Aspect ratio to make the sphere look more like a sphere
        aspectmode='data' # 'auto', 'cube', 'data', 'manual'
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    showlegend=True
)

# Show the plot
fig.show()

## 6. How about 5-Vs?

In this section, values along 5Vs are provided.

Thus, **All-mighty embedding is unnecessary to call**.

Below is a typical preception with inital training/ While erros exist across V2/V3/V4/V5, the student can vaguely see the correct answer Bv from the wrong choices Ax, Cx, and Dx.

5Vs---Pa---P1---P2---P3---Q?---Av---Bx---Cx---Dx---

V1-----1-----1-----1------1-----1------1-----1-----1-----0---

V2-----1-----1-----1-----1------1-----1------1------1-----1---

V3-----0-----0-----0-----0-----0------0------0------0-----0---

V4-----0-----0-----0-----0-----0------0------0------1-----0---

V5-----0-----0-----0-----0-----0------0------1------0-----1---

In [22]:
# Scatterplot and P-Tet: 5V-ini
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.decomposition import PCA

# Your data as a string (including header row)
data_string = """datapoint,V1,V2,V3,V4,V5
Pa,1,1,0,0,0
P1,1,1,0,0,0
P2,1,1,0,0,0
P3,1,1,0,0,0
Q?,1,1,0,0,0
Av,1,1,0,0,0
Bx,1,1,0,0,1
Cx,1,1,0,1,0
Dx,9,1,0,0,1
"""

# Load the data from the string into a Pandas DataFrame
from io import StringIO
df = pd.read_csv(StringIO(data_string))

# Extract the data for PCA (exclude the 'datapoint' column)
X = df.iloc[:, 1:].values  # Get values from columns V1 to V5

# Extract the datapoint labels
labels = df['datapoint'].tolist()

# Perform PCA
pca = PCA(n_components=3)
reduced_data = pca.fit_transform(X)

# Get the indices of P1, P2, P3, and Q?
p1_index = labels.index('P1')
p2_index = labels.index('P2')
p3_index = labels.index('P3')
q_index = labels.index('Q?')

# Extract the 3D coordinates of P1, P2, P3, and Q?
p1_coords = reduced_data[p1_index]
p2_coords = reduced_data[p2_index]
p3_coords = reduced_data[p3_index]
q_coords = reduced_data[q_index]

# Define the vertices of the tetrahedron
tetrahedron_vertices = np.array([p1_coords, p2_coords, p3_coords, q_coords])

# Define the edges of the tetrahedron (indices of vertices)
tetrahedron_edges = [
    (0, 1), (0, 2), (0, 3),  # Edges from P1 to P2, P3, Q?
    (1, 2), (1, 3),          # Edges from P2 to P3, Q?
    (2, 3)                   # Edge from P3 to Q?
]

# Create the 3D scatter plot for all points
scatter = go.Scatter3d(
    x=reduced_data[:, 0],
    y=reduced_data[:, 1],
    z=reduced_data[:, 2],
    mode='markers+text',
    text=labels,
    textposition="middle right",
    marker=dict(size=4),
    name='Data Points'  # Add a name for the scatter plot
)

# Create the lines for the tetrahedron edges
lines = []
for i, (start_index, end_index) in enumerate(tetrahedron_edges):
    start_point = tetrahedron_vertices[start_index]
    end_point = tetrahedron_vertices[end_index]
    lines.append(
        go.Scatter3d(
            x=[start_point[0], end_point[0]],
            y=[start_point[1], end_point[1]],
            z=[start_point[2], end_point[2]],
            mode='lines',
            line=dict(color='red', width=4),  # Style the lines
            name=f'Edge {i+1}' # Add a name for each line.
        )
    )

# Combine scatter and lines
data = [scatter] + lines

# Set the title and axis labels
layout = go.Layout(
    title="Scatterplot and P-Tet: 5V-ini",
    scene=dict(
        xaxis_title="PC 1",
        yaxis_title="PC 2",
        zaxis_title="PC 3",
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    showlegend=True #show legend
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Show the plot (this will display an interactive plot in Colab)
fig.show()
